In [2]:
from datasets import load_from_disk

preprocessed_dataset_path = r"C:\Users\Debajyoti\OneDrive\Desktop\Generative_AI_(fine_tuned_model__detoxify_summarization\data\preprocess_datasets"
dataset = load_from_disk(preprocessed_dataset_path)
print("Datyaset loaded successfully!")

Datyaset loaded successfully!


In [3]:
from sentiment_model import EvaluateToxicity,toxicity_model,toxicity_tokenizer
from transformers import AutoTokenizer
from create_ppo_and_ref_model import ref_model



In [4]:
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, device_map="auto")

evaluate_toxicity = EvaluateToxicity(
    toxicity_classifier_model=toxicity_model,
    main_model=ref_model,
    toxicity_tokenizer=toxicity_tokenizer,
    main_tokenizer=tokenizer,
    dataset=dataset["train"],
    num_samples=10
)

mean_before_detoxification, std_before_detoxification = evaluate_toxicity.evaluate_mean_and_variance()

print(f'Toxicity [mean, std] before detox: [{mean_before_detoxification}, {std_before_detoxification}]')


10it [02:10, 13.07s/it]

Toxicity [mean, std] before detox: [0.02238751995464554, 0.05002900902694688]


In [5]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

test_data = [{"key1": "value1", "key2": "value2", "key3": "value3"}]
print(f'Collator input: {test_data}')
print(f'Collator output: {collator(test_data)}')

Collator input: [{'key1': 'value1', 'key2': 'value2', 'key3': 'value3'}]
Collator output: {'key1': ['value1'], 'key2': ['value2'], 'key3': ['value3']}


In [6]:
from trainer import ppo_trainer, fine_tuning_instruct_model

output_min_length = 100
output_max_length = 400
max_ppo_epoch = 10

ppo_trainer,ppo_model = fine_tuning_instruct_model(
    output_min_length=output_min_length,
    output_max_length = output_max_length,
    ppo_trainer = ppo_trainer,
    evaluate_toxicity=evaluate_toxicity,
    max_ppo_epoch=max_ppo_epoch
)
print("Successfully Training copmpleted!")



c:\Users\Debajyoti\AppData\Local\Programs\Python\Python312\Lib\site-packages\trl\trainer\ppo_trainer.py:193: FutureWarning: `PPOTrainer` is deprecated and will be removed in trl v0.12. Please use `PPOv2Trainer` instead.
  warnings.warn(
0it [00:00, ?it/s]Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
1it [00:23, 23.84s/it]

objective/kl: 0.007212568074464798
ppo/returns/mean: 0.06715904176235199
ppo/policy/advantages_mean: 0.005440084263682365
----------------------------------------------------------------------------------------------------


2it [00:46, 23.33s/it]

objective/kl: -0.005001811310648918
ppo/returns/mean: 0.08179621398448944
ppo/policy/advantages_mean: -0.10700567066669464
----------------------------------------------------------------------------------------------------


3it [01:03, 20.41s/it]

...

objective/kl: -0.05809370428323746
ppo/returns/mean: 0.061154626309871674
ppo/policy/advantages_mean: -0.03513656556606293
----------------------------------------------------------------------------------------------------


9it [02:54, 19.30s/it]

objective/kl: -0.005539754405617714
ppo/returns/mean: 0.05540284886956215
ppo/policy/advantages_mean: -0.04244184494018555
----------------------------------------------------------------------------------------------------


10it [03:11, 19.11s/it]

objective/kl: 0.0557577908039093
ppo/returns/mean: 0.043449848890304565
ppo/policy/advantages_mean: 0.0010683070868253708
----------------------------------------------------------------------------------------------------
Successfully Training copmpleted!


In [20]:
import pickle
import os

def save_ppo_model_and_trainer_with_pickle(ppo_model, path="ppo_model_trainer.pkl"):
    os.makedirs(os.path.dirname(path), exist_ok=True)

    try:

        save_dict = {
            'ppo_trained_model': ppo_model.state_dict()                  
        }

        with open(path, 'wb') as f:
            pickle.dump(save_dict, f)

        print(f"PPO Model and PPO Trainer saved successfully at {path}")
    
    except Exception as e:
        print(f"Error while saving PPO model and trainer: {e}")


In [21]:
path = r"C:\Users\Debajyoti\OneDrive\Desktop\Generative_AI_(fine_tuned_model__detoxify_summarization\models\ppo_trained_models.pkl"
save_ppo_model_and_trainer_with_pickle(ppo_model,path)

PPO Model and PPO Trainer saved successfully at C:\Users\Debajyoti\OneDrive\Desktop\Generative_AI_(fine_tuned_model__detoxify_summarization\models\ppo_trained_models.pkl


In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the correct device
ppo_model = ppo_model.to(device)
toxicity_model = toxicity_model.to(device)

# Move dataset to the correct device
def move_to_device(batch, device):
    if isinstance(batch, dict):
        return {k: move_to_device(v, device) for k, v in batch.items()}
    elif isinstance(batch, list):
        return [move_to_device(v, device) for v in batch]
    return batch.to(device)

dataset["train"] = dataset["train"].map(lambda batch: move_to_device(batch, device), batched=True)

# Ensure the tokenizer uses the correct device for input tensors
evaluate_toxicity_after_training = EvaluateToxicity(
    toxicity_classifier_model=toxicity_model,
    main_model=ppo_model,
    toxicity_tokenizer=toxicity_tokenizer,
    main_tokenizer=tokenizer,
    dataset=dataset["train"],
    num_samples=10
)

mean_after_detoxification, std_after_detoxification = evaluate_toxicity_after_training.evaluate_mean_and_variance()


In [ ]:

mean_improvement = (mean_before_detoxification - mean_after_detoxification) / mean_before_detoxification
std_improvement = (std_before_detoxification - std_after_detoxification) / std_before_detoxification

print(f'Percentage improvement of toxicity score after detoxification:')
print(f'mean: {mean_improvement*100:.2f}%')
print(f'std: {std_improvement*100:.2f}%')

In [29]:
import torch
from trl.core import LengthSampler
from tqdm import tqdm

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# LengthSampler setup
output_min_length = 100
output_max_length = 400
output_length_sampler = LengthSampler(output_min_length, output_max_length)

generation_kwargs = {
    "min_length": 5,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True
}

def predict_test_data(ref_model, ppo_model, dataset,tokenizer):
    batch_size = 20
    compare_results = {}

    # Select the batch of data
    df_batch = dataset["test"][0:batch_size]

    compare_results["query"] = df_batch["query"]
    prompt_tensors = df_batch["input_ids"]

    # Move models to the correct device
    ref_model = ref_model.to(device)
    ppo_model = ppo_model.to(device)

    summary_tensors_ref = []
    summary_tensors_ppo = []

    # Process each sample in the batch
    for i in tqdm(range(batch_size)):
        gen_len = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len

        # Move the input tensor to the correct device
        prompt_tensor = torch.as_tensor(prompt_tensors[i]).unsqueeze(dim=0).to(device)

        # Generate summary from the reference model
        summary_ref = ref_model.generate(
            input_ids=prompt_tensor,
            **generation_kwargs
        ).squeeze()[-gen_len:]
        summary_tensors_ref.append(summary_ref)

        # Generate summary from the PPO model
        summary_ppo = ppo_model.generate(
            input_ids=prompt_tensor,
            **generation_kwargs
        ).squeeze()[-gen_len:]
        summary_tensors_ppo.append(summary_ppo)

    compare_results["response_before"] = [tokenizer.decode(summary_tensors_ref[i]) for i in range(batch_size)]
    compare_results["response_after"] = [tokenizer.decode(summary_tensors_ppo[i]) for i in range(batch_size)]
    return compare_results


In [30]:
compare_results = predict_test_data(ref_model=ref_model,
                                    ppo_model = ppo_model,
                                    dataset = dataset,
                                    tokenizer = tokenizer)

print("Execution successfully completed")

100%|██████████| 20/20 [00:27<00:00,  1.37s/it]

Execution successfully completed


In [31]:
print(compare_results)

{'query': ["Summarize the following conversation: #Person1#: Good afternoon doctor. #Person2#: Good afternoon, Mrs. Brown. Wow, what's wrong with this little boy? #Person1#: He is my son Jim. He's gotta cough. #Person2#: How long has he been like this? #Person1#: Ever since last night. #Person2#: Has he had anything to eat today? #Person1#: Yes, he had a little milk and an egg this morning. #Person2#: Well, Mrs. Brown. I think he's caught a cold. #Person1#: Is it serious? #Person2#: No, it's nothing serious, but he better stay at home and rest. #Person1#: Thank you very much. #Person2#: You're welcome. Summary:", "Summarize the following conversation: #Person1#: Let me tell you what happened to my homework. #Person2#: OK, go ahead--what is the excuse THIS time? #Person1#: Actually, I did it, but then it got lost. #Person2#: Could you have gotten it done at another time? #Person1#: Yes, I could. #Person2#: You have never missed an assignment before--when will you be making this one up? 

In [32]:
for key in compare_results.keys():
    print(key)

query
response_before
response_after


In [33]:
import pandas as pd

test_result_path = r"C:\Users\Debajyoti\OneDrive\Desktop\Generative_AI_(fine_tuned_model__detoxify_summarization\results_csv_file\final_comparison_result.csv"

# Save the comparison results to CSV
result_df = pd.DataFrame(compare_results)
result_df.to_csv(test_result_path, index=False)

# Print success message
print(f"CSV file sent to destined path: {test_result_path}")


CSV file sent to destined path: C:\Users\Debajyoti\OneDrive\Desktop\Generative_AI_(fine_tuned_model__detoxify_summarization\results_csv_file\final_comparison_result.csv


In [34]:
result_df.head()


,query,response_before,response_after
0,Summarize the following conversation: #Person1...,<pad> Jim's cough is always suspected.</s>,<pad> The little boy is doing OK today.</s>
1,Summarize the following conversation: #Person1...,<pad> #Person1: I skipped my homework. #Person...,<pad> Problem solved. Learn who or do it to ma...
2,Summarize the following conversation: #Person1...,<pad> Employees receive allowance of $50 a wee...,<pad> She spoke about their benefits.</s>
3,Summarize the following conversation: #Person1...,<pad> The yard sale on Saturday morning is at ...,<pad> Whole8% every time we see what brands we...
4,Summarize the following conversation: #Person1...,<pad> When does trouble start; how does it get...,"<pad> Practice taking these drugs, which work ..."
